In [3]:
# Setup

# Install packages
!pip install smartapi-python
!pip install logzero
!pip install pyotp
!pip install websocket-client
!pip install ijson
!pip install mplfinance
!pip install gspread google-auth
!pip install pandas plotly

url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!pip install conda-package-handling
!wget https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
!cph x ta-lib-0.5.1-py311h9ecbd09_0.conda
!mv ./ta-lib-0.5.1-py311h9ecbd09_0/lib/python3.11/site-packages/talib /usr/local/lib/python3.11/dist-packages/

# Set time zone to IST
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Kolkata /etc/localtime
!date


In [4]:

# package import statement
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
import datetime
import zipfile
import os
from datetime import time, timedelta, datetime
import ijson
import glob
import json
import mplfinance as mpf
import time
import plotly.graph_objects as go
import requests
import re
import pandas as pd
from talib.abstract import *

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [28]:
import pandas as pd
import re
from datetime import datetime

def data_cleaning(file_name):
    date_str = file_name.name.split("_")[-1].split(".")[0]  # Extract 'DDMMYYYY'
    target_date = pd.to_datetime(date_str, format="%d%m%Y")  # Convert once

    print("\n--------------------------------------")
    print(f"Date: {target_date.strftime('%d %b %Y')}")

    df = pd.read_csv(file_name, usecols=['Ticker', 'Time', 'Open', 'High', 'Low', 'Close'], engine='python')  # Load only required columns
    df['Time'] = pd.to_datetime(df['Time'].str.strip(), format="%H:%M:%S").dt.time  # Vectorized conversion

    # Filter required tickers directly
    mask = df['Ticker'].str.startswith('NIFTY') & df['Ticker'].str.endswith(('PE.NFO', 'CE.NFO'))
    df = df.loc[mask]

    # Extract expiry date using vectorized regex
    date_pattern = df['Ticker'].str.extract(r'(\d{2})([A-Z]{3})(\d{2})')
    if date_pattern.isnull().any().any():
        print("Warning: Some tickers did not match the expected pattern.")

    date_pattern.dropna(inplace=True)
    date_pattern.columns = ['Day', 'Month', 'Year']

    # Convert expiry dates
    date_pattern['Year'] = date_pattern['Year'].astype(int) + 2000
    date_pattern['Day'] = date_pattern['Day'].astype(int)
    date_pattern['Month'] = pd.to_datetime(date_pattern['Month'], format='%b').dt.month  # Vectorized month conversion
    df['Expiry_Date'] = pd.to_datetime(date_pattern[['Year', 'Month', 'Day']])  # Efficient date creation

    # Find the closest expiry date
    min_expiry_date = df['Expiry_Date'][df['Expiry_Date'] >= target_date].min()
    df = df.loc[df['Expiry_Date'] == min_expiry_date]

    df = df.drop(columns=['Expiry_Date'])

    return target_date, df


def add_time_to_string(time_str, m):
    # Parse the time string into a datetime object
    time_obj = datetime.strptime(time_str, "%H:%M:%S")

    # Add the specified hours and minutes
    new_time_obj = time_obj + timedelta(minutes=m)

    # Format the new time back to a string
    new_time_str = new_time_obj.strftime("%H:%M:%S")

    return new_time_str

def find_nearest_options(df, price_input, curr_time):

    # Extract hour and minute from x
    curr_time = pd.to_datetime(curr_time, format='%H:%M:%S')

    # Filter rows where the hour and minute match
    df_filtered = df[(df['Time'].apply(lambda x: x.hour) == curr_time.hour) &
                  (df['Time'].apply(lambda x: x.minute) == curr_time.minute)].copy()

    # Find difference
    df_filtered.loc[:, 'Closest'] = abs(df_filtered['Close'] - price_input)

    # Separate CE and PE options (using endswith)
    ce_options = df_filtered[df_filtered['Ticker'].str.endswith('CE.NFO')]
    pe_options = df_filtered[df_filtered['Ticker'].str.endswith('PE.NFO')]

    # Find the first CE and PE with the smallest Close >= price_input
    nearest_ce = ce_options.loc[ce_options['Closest'].idxmin()] if not ce_options.empty else None
    nearest_pe = pe_options.loc[pe_options['Closest'].idxmin()] if not pe_options.empty else None

    return nearest_ce, nearest_pe

def sl_and_reentry(df, ticker, price, time):

    # Initialize variables
    stop_loss_price = 60  # Stop loss threshold
    reentry_price = 40    # Re-entry threshold
    profit_loss = 0       # Track profit/loss
    curr_time = time      # Start tracking from the given time
    profit_p = []
    curr_price = price
    transactions = []

    while True:

        # Extract hour and minute from x
        df_filtered = df[(df['Ticker'] == ticker)].copy()
        curr_time = pd.to_datetime(curr_time, format='%H:%M:%S')


        # Assuming curr_time is a datetime.time object
        df_filtered = df_filtered[
            (
                (df_filtered['Time'].apply(lambda x: x.hour) > curr_time.hour)
            ) |
            (
                (df_filtered['Time'].apply(lambda x: x.hour) == curr_time.hour) &
                (df_filtered['Time'].apply(lambda x: x.minute) > curr_time.minute)
            )
        ]

        # Filter by price
        df_filtered_price = df_filtered[(df_filtered['High'] >= stop_loss_price)].copy()

        if (len(df_filtered_price) == 0): # No SL hit
            df_filtered = df_filtered.sort_values(by='Time')
            profit_p.append((- df_filtered.iloc[-1]['Close'] + curr_price) / 200000 * 100 * 75)
            transactions.append((curr_time, curr_price, df_filtered.iloc[-1]['Close'])) # (sell, buy)
            curr_time = df_filtered.iloc[-1]['Time']
        else: # SL
            # Sort by Time and get the first row
            reentry_price = curr_price
            result = df_filtered_price.sort_values(by='Time')
            first_r = result.iloc[0]
            profit_p.append((-stop_loss_price + curr_price) / 200000 * 100 * 75)
            transactions.append((curr_time, curr_price, stop_loss_price)) # (sell, buy)
            curr_time = first_r['Time']

        return curr_time, profit_p, transactions


    return None

def set_yellow(sh, next_row):

    worksheet = sh.get_worksheet(0)  # Get the first worksheet

     # Format the row yellow
    format_body = {
        "requests": [{
            "repeatCell": {
                "range": {
                    "sheetId": worksheet.id,
                    "startRowIndex": next_row - 1,  # gspread uses 0-based indexing
                    "endRowIndex": next_row,
                    "startColumnIndex": 0,
                    "endColumnIndex": 26  # Assuming 26 columns like before
                },
                "cell": {
                    "userEnteredFormat": {
                        "backgroundColor": {
                            "red": 1.0, "green": 1.0, "blue": 0.0  # Yellow
                        }
                    }
                },
                "fields": "userEnteredFormat.backgroundColor"
            }
        }]
    }

    # Apply the format directly using the format_body
    # The format_body is already in the correct structure for batch_update
    sh.batch_update(format_body)

def get_sh():
    sheet_url = "https://docs.google.com/spreadsheets/d/17KL8LiAjjDZZXFKkhGhCugIqLe-C8NCGB97EQqLvZZ4/edit?usp=sharing"
    sh = gc.open_by_url(sheet_url)
    return sh

def backtest(target_date, df, display):

    curr_time = "09:30:00"
    active = set()
    profit = 0
    profit_p = []
    price_input = 40
    profit_track = []

    sh = get_sh()
    worksheet = sh.get_worksheet(0)  # Get the first worksheet
    headers = ['Time', 'Buy', 'Sell']  # List of column names
    worksheet.update('A1', [headers])  # Update row 1 with your headers

    # Get the number of rows in the sheet
    all_values = worksheet.get_all_values()
    next_row = len(all_values) + 1 if all_values else 1

    # Determine the range for the next row. Assuming 26 columns (A to Z)
    cells = worksheet.range(f'A{next_row}:Z{next_row}')

    # Convert pandas.Timestamp to string
    target_date = target_date.strftime("%Y-%m-%d")

    # Update the first column with the date
    cell = worksheet.cell(next_row, 1)  # Column 1 is 'A'
    cell.value = target_date
    worksheet.update_cells([cell])
    next_row += 1

    while curr_time <= "15:00:00":

        i, j = 0, 0
        curr = find_nearest_options(df, price_input, curr_time)
        new_time = add_time_to_string(curr_time, 30)

        if curr[0] is not None:

            if display:
                cell = worksheet.cell(next_row, 1)  # Column 1 is 'A'
                cell.value = "Call"
                worksheet.update_cells([cell])
                next_row += 1

            c_ticker = curr[0]['Ticker']
            c_price = curr[0]['Close']
            x_time = curr_time
            while i < 3:
                if x_time <= "15:00:00":
                    x_time, x_profit, x_transactions = sl_and_reentry(df, c_ticker, c_price, x_time)
                    x_time = str(x_time)[-8:]
                    curr1 = find_nearest_options(df, price_input, x_time)
                    profit_p.extend(x_profit)

                    c_ticker = curr1[0]['Ticker']
                    c_price = curr1[0]['Close']


                    if display:
                        # Convert to the format you want for the time column (HH:MM:SS)
                        time_only = x_transactions[0][0].time().strftime("%H:%M:%S")
                        buy, sell = x_transactions[0][1], x_transactions[0][2]
                        # Insert transaction data
                        range_str = f'A{next_row}:C{next_row}'  # A for time, B for buy, C for sell
                        worksheet.update(range_str, [[time_only, buy, sell]])
                        next_row += 1

                    i += 1
                else:
                    break

        if curr[1] is not None:

            if display:
                cell = worksheet.cell(next_row, 1)  # Column 1 is 'A'
                cell.value = "Put"
                worksheet.update_cells([cell])
                next_row += 1

            p_ticker = curr[1]['Ticker']
            p_price = curr[1]['Close']

            x_time = curr_time
            while j < 3:
                if x_time <= "15:00:00":
                    x_time, x_profit, x_transactions = sl_and_reentry(df, p_ticker, p_price, x_time)
                    x_time = str(x_time)[-8:]
                    curr2 = find_nearest_options(df, price_input, x_time)
                    profit_p.extend(x_profit)

                    p_ticker = curr2[1]['Ticker']
                    p_price = curr2[1]['Close']

                    if display:
                        # Convert to the format you want for the time column (HH:MM:SS)
                        time_only = x_transactions[0][0].time().strftime("%H:%M:%S")
                        buy, sell = x_transactions[0][1], x_transactions[0][2]
                        # Get the number of rows in the sheet
                        all_values = worksheet.get_all_values()
                        # Insert transaction data
                        range_str = f'A{next_row}:C{next_row}'  # A for time, B for buy, C for sell
                        worksheet.update(range_str, [[time_only, buy, sell]])
                        next_row += 1

                    j += 1

                else:
                    break

        curr_time = new_time


    # Initialize variables
    total_sum = sum(profit_p)  # Sum of all values
    num_win_days = sum(1 for value in profit_p if value > 0)  # Count of positive values
    num_loss_days = sum(1 for value in profit_p if value < 0)  # Count of negative values

    # Print results in sheets
    sh = get_sh()
    worksheet = sh.get_worksheet(0)
    all_values = worksheet.get_all_values()
    next_row = len(all_values) + 1 if all_values else 1
    range_str = f'A{next_row}:C{next_row}'
    worksheet.update(range_str, [[total_sum, num_win_days, num_loss_days]])

    # Print results
    print(profit_p)
    print("Total Profit %:", total_sum)
    print("Number of wins:", num_win_days)
    print("Number of losses:", num_loss_days)

    return total_sum


def process_nested_zip(zip_file_path):
    """
    Processes a zip file containing monthly zip files, which in turn contain CSV files.

    Args:
    zip_file_path: The path to the main zip file (e.g., "2020.zip").
    """

    sh = get_sh()
    worksheet = sh.get_worksheet(0)  # Get the first worksheet
    worksheet.clear()

    i = 0
    total = 0
    with zipfile.ZipFile(zip_file_path, 'r') as main_zip:
        for month_zip_info in main_zip.infolist():
            if month_zip_info.filename.endswith(".zip"):  # Process only zip files
                month_zip_name = month_zip_info.filename
                with main_zip.open(month_zip_info) as month_zip_file:
                    if "__MACOSX" in month_zip_info.filename or month_zip_info.filename.startswith("._"):
                        continue  # Skip macOS metadata files
                    with zipfile.ZipFile(month_zip_file, 'r') as month_zip:
                        for csv_file_info in month_zip.infolist():
                            if csv_file_info.filename.endswith(".csv"):
                                csv_file_name = csv_file_info.filename
                                with month_zip.open(csv_file_info) as csv_file:
                                    # Assuming your backtest function takes a file path as input
                                    if "__MACOSX" in csv_file_info.filename or csv_file_info.filename.startswith("._"):
                                        continue
                                    sh = get_sh()
                                    worksheet = sh.get_worksheet(0)  # Get the first worksheet
                                    if i < 2:
                                        print = True
                                    elif i == 2:
                                        sh = get_sh()
                                        worksheet = sh.get_worksheet(0)
                                        all_values = worksheet.get_all_values()
                                        next_row = len(all_values) + 1 if all_values else 1
                                        range_str = f'A{next_row}:C{next_row}'
                                        worksheet.update(range_str, [["Profit", "Win Days", "Loss Days"]])

                                    i += 1
                                    target_date, df = data_cleaning(csv_file)
                                    total += backtest(target_date, df, print)
                                    print = False


    print("Total : ", total)

# Call the function to process your "2020.zip" file
process_nested_zip("/content/2020.zip")





--------------------------------------
Date: 02 Nov 2020


<ipython-input-28-ea68c0dd2dea>:175: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', [headers])  # Update row 1 with your headers
<ipython-input-28-ea68c0dd2dea>:227: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(range_str, [[time_only, buy, sell]])
<ipython-input-28-ea68c0dd2dea>:227: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(range_str, [[time_only, buy, sell]])
<ipython-input-28-ea68c0dd2dea>:264: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_na

[-0.808125, 0.26249999999999996, 0.5456249999999998, -0.703125, -0.035624999999999844, 0.5924999999999999, -0.8943750000000001, 0.26249999999999996, 0.42749999999999994, -0.9281250000000001, 0.26249999999999996, 0.34312499999999996, -0.8174999999999999, 0.26249999999999996, 0.31124999999999986, -0.541875, -0.035624999999999844, 0.44625, -0.643125, -0.035624999999999844, 0.49125, -0.9299999999999998, 0.26249999999999996, 0.29249999999999987, -0.5625, -0.035624999999999844, 0.3, -0.9356250000000002, 0.26249999999999996, 0.290625, 0.32812499999999994, -0.16500000000000023, 0.27187500000000003, -0.3637500000000001]
Total Profit %: -2.22
Number of wins: 18
Number of losses: 16

--------------------------------------
Date: 03 Nov 2020


<ipython-input-28-ea68c0dd2dea>:175: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', [headers])  # Update row 1 with your headers
<ipython-input-28-ea68c0dd2dea>:227: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(range_str, [[time_only, buy, sell]])
<ipython-input-28-ea68c0dd2dea>:264: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(range_str, [[time_only, buy, sell]])
<ipython-input-28-ea68c0dd2dea>:227: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_na

[-0.5456249999999998, 0.075, -0.7387499999999998, 0.12562500000000004, -0.6299999999999998, 0.0393749999999999, -0.3674999999999999, 0.0037499999999997874, -0.455625, 0.0037499999999997874, -0.4856249999999998, 0.04499999999999984, -0.5549999999999999, 0.046875, -0.26437499999999986, -0.09562500000000017, -0.4893749999999999, 0.0, -0.47999999999999987, -0.1181250000000002, -0.71625, 0.0393749999999999, -0.41812499999999997, -0.15]
Total Profit %: -6.13125
Number of wins: 8
Number of losses: 15


...

[0.17624999999999996, -0.511875, 0.26249999999999996, 0.21750000000000003, -0.07500000000000014, 0.301875, -0.44437500000000013, 0.26249999999999996, 0.18375000000000005, 0.024374999999999945, 0.17812499999999987, 0.009375, -0.1875, 0.26249999999999996, -0.0637500000000001, 0.13874999999999985, -0.022500000000000055, 0.14437499999999978, -0.09937500000000021, 0.20437499999999983, -0.084375, 0.16687499999999983, -0.2681250000000002, 0.2568749999999998, -0.3, 0.17062499999999992]
Total Profit %: 0.9037499999999979
Number of wins: 16
Number of losses: 10

--------------------------------------
Date: 06 Jan 2020


<ipython-input-28-ea68c0dd2dea>:175: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', [headers])  # Update row 1 with your headers
<ipython-input-28-ea68c0dd2dea>:286: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(range_str, [[total_sum, num_win_days, num_loss_days]])


[0.9674999999999999, 0.10124999999999984, 1.12875, -0.6975000000000001, -0.14624999999999994, 1.0481250000000002, -0.680625, -0.14624999999999994, 0.9075000000000001, -0.496875, -0.14624999999999994, 0.7725, -0.32062499999999994, 0.67125, -0.28125, 0.6299999999999998, -0.22124999999999992, 0.5437500000000001, -0.2418750000000001, 0.44625, -0.1575000000000001, 0.2981249999999998, -0.1293750000000001, 0.15937500000000002, 0.07875000000000006, 0.27187500000000003, -0.1762500000000001]
Total Profit %: 4.183124999999999
Number of wins: 14
Number of losses: 13

--------------------------------------
Date: 07 Jan 2020


<ipython-input-28-ea68c0dd2dea>:175: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', [headers])  # Update row 1 with your headers


KeyboardInterrupt: 